# Если вы используется GoogleColab

Не забудьте сменить среду выполнения на **GPU** - так программа будет работать быстрее.

Если вам удобнее загрузить данные непосредственно в сессионное хранилище, то ничего выполнять не нужно.

Если вам удобнее загрузить данные на гугл диск, то выполните следующие ячейки:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive
# эта ячейка делает директорией по умолчанию ту директорию, в которой у вас расположены все файлы
# или, говоря иначе, то место, в которое вы попадаете при открытии диска
# если все данные программы лежат в какой-то папке под названием directory_name
# то используется следующую команду: cd drive/MyDrive/directory_name

# Библиотеки и дополнительные функции

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import numpy as np

import torch
import torch.nn as nn
from torchvision.models import vgg13
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm.notebook import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.transforms import Compose
from torchvision import transforms
import torchvision.transforms as T
import torchvision.transforms.functional as TTF


from PIL import Image, ImageEnhance
import cv2 as cv
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage import binary_fill_holes as fill_holes
from skimage.measure import moments_coords
from skimage.measure import label, find_contours, regionprops

In [4]:
def angle_between(vec1, vec2):
    """ 
    Угол между двумя векторами (в градусах)
    """
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)
    return np.arccos(np.clip(np.dot(vec1_unit, vec2_unit), -1, 1)) * 57.29

# Реализация свёрточной нейронной сети UNet

In [5]:
class VGG13Encoder(torch.nn.Module):
    def __init__(self, num_blocks, pretrained=True):
        super().__init__()
        self.num_blocks = num_blocks
        self.blocks = nn.ModuleList()
        # Obtaining pretrained VGG model from torchvision.models and
        # copying all layers except for max pooling.
        feature_extractor = vgg13(pretrained=pretrained).features
        for i in range(self.num_blocks):
            self.blocks.append(
                torch.nn.Sequential(*[
                    feature_extractor[j]
                    for j in range(i * 5, i * 5 + 4)
                ])
            )

    def forward(self, x):
        activations = []
        for i in range(self.num_blocks):
            x = self.blocks[i](x)
            activations.append(x)
            if i != self.num_blocks - 1:
                x = torch.functional.F.max_pool2d(x, kernel_size=2, stride=2)
        return activations

In [6]:
class DecoderBlock(torch.nn.Module):
    def __init__(self, out_channels):
        super().__init__()

        self.upconv = torch.nn.Conv2d(
            in_channels=out_channels * 2, out_channels=out_channels,
            kernel_size=3, padding=1, dilation=1
        )
        self.conv1 = torch.nn.Conv2d(
            in_channels=out_channels * 2, out_channels=out_channels,
            kernel_size=3, padding=1, dilation=1
        )
        self.conv2 = torch.nn.Conv2d(
            in_channels=out_channels, out_channels=out_channels,
            kernel_size=3, padding=1, dilation=1
        )
        self.relu = nn.ReLU()
        
    def forward(self, down, left):
        x = torch.nn.functional.interpolate(down, scale_factor=2)
        x = self.upconv(x)
        x = self.relu(self.conv1(torch.cat([left, x], 1)))
        x = self.relu(self.conv2(x))
        return x

In [7]:
class Decoder(nn.Module):
    def __init__(self, num_filters, num_blocks):
        super().__init__()

        for i in range(num_blocks):
            self.add_module(f'block{num_blocks - i}', DecoderBlock(num_filters * 2**i))

    def forward(self, acts):
        up = acts[-1]
        for i, left in enumerate(acts[-2::-1]):
            up = self.__getattr__(f'block{i + 1}')(up, left)
        return up

In [8]:
class UNet(torch.nn.Module):
    def __init__(self, num_classes=1, num_filters=64, num_blocks=4):
        super().__init__()
        self.encoder = VGG13Encoder(num_blocks=num_blocks)
        self.decoder = Decoder(num_filters=64, num_blocks=num_blocks - 1)
        self.final = torch.nn.Conv2d(
            in_channels=num_filters, out_channels=num_classes, kernel_size=1
        )

    def forward(self, x):
        acts = self.encoder(x)
        x = self.decoder(acts)
        x = self.final(x)
        return x

# Основная функция, выдающая результат работы программы

In [9]:
def predict(model, path, filename, threshold=0.1):
  # прочитаем изображение 
  image = cv.imread(path + '/' + filename)

  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)
  model.eval()

  # произведем все необходимые трансформации
  mean = [0.485, 0.456, 0.406]
  std = [0.229, 0.224, 0.225]
  totensor = transforms.ToTensor()
  img = totensor(image)
  img = F.pad(img, (1, 1, 1, 1))
  img = TTF.resize(img, [704, 512])
  img = img.unsqueeze(0)
  normalize = transforms.Normalize(mean, std)
  img = normalize(img)

  img = img.to(device)
  with torch.no_grad():
    out = model(img).cpu()
  
  # получим результат работы нейросети
  res = torch.sigmoid(out)

  # получим маску, путем бинаризации
  a = np.array(res.squeeze())
  a[a < threshold] = 0
  a[a >= threshold] = 255
  a = a.astype(np.uint8)

  # найдём все необходимые контуры в маске (их должно быть 9)
  contours, hierarchy = cv.findContours(a, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

  # определим центры контуров, как центры масс
  centers = []
  for contour in contours:
    if len(contour) >= 5:
      M = moments_coords(contour.squeeze())
      center = int(M[1, 0]/M[0, 0]), int(M[0, 1]/M[0, 0])
      centers.append(center)

  if len(centers) != 9:
    print('Для изображения ' + name + ' не удалось корректно определить маску.')
    return -1
  
  # переведем исходное изображение в черно-белое и отфильтруем
  if image.shape != (702, 510, 3):
    pil_image = Image.open(path + '/' + filename).convert('RGB') 
    pil_image = TTF.resize(pil_image, [704, 512])
    open_cv_image = np.array(pil_image) 
    # Convert RGB to BGR 
    image = open_cv_image[:, :, ::-1].copy() 
  image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
  image_gray = cv.cvtColor(image_rgb, cv.COLOR_RGB2GRAY)
  _, image_bin = cv.threshold(image_gray, 90, 255, cv.THRESH_BINARY)

  # найдём центр руки
  labeled_image = label(image_bin) 
  for component in regionprops(labeled_image): 
    if component.area > 35000: 
      center_ruki = tuple(map(int, component.centroid))
      center_ruki = (center_ruki[1], center_ruki[0])
  
  # найдем расстояния между центром руки и всеми 9 точками
  distances = []
  for center in centers:
    dist = np.sqrt((center[0] - center_ruki[0])**2 + (center[1] - center_ruki[1])**2)
    distances.append(dist)
  
  distances = np.array(distances)
  osnov = np.argsort(distances)[:4] # это номера перемычек в списке centers
  paltsi = [] # это координаты пальцев
  for i in np.argsort(distances)[4:]:
    paltsi.append(centers[i])
  
  hands_bases = [] # это координаты перемычек
  for i in osnov:
    hands_bases.append(centers[i])
  
  # найдём попарные расстояния между всеми перемычками
  base_dists = []
  for base in hands_bases:
    tmp = []
    for base_ in hands_bases:
      dist = np.sqrt((base[0] - base_[0])**2 + (base[1] - base_[1])**2)
      if dist != 0:
        tmp.append((dist, base, base_))
    base_dists.append(tmp)
  
  # максимальное из этих расстояний будет соответствовать большой перемычке
  maximum = 0
  big_base = None
  for i in base_dists:
    tmp = 0
    for j in i:
      tmp += j[0]
    if tmp > maximum:
      maximum = tmp
      big_base = j[1]
      big_base_dists = i
  
  base_dists = []
  for base in hands_bases:
    dist = np.sqrt((big_base[0] - base[0])**2 + (big_base[1] - base[1])**2)
    if dist != 0:
      base_dists.append((dist, base))

  base_dict = {} # это словарь перемычек
  base_dict[0] = big_base 
  for i, j in enumerate(sorted(base_dists)):
    base_dict[i+1] = j[1]
  
  # находим большой палец
  dists = []
  for palets in paltsi:
    dist = np.sqrt((base_dict[0][0] - palets[0])**2 + (base_dict[0][1] - palets[1])**2)
    dists.append((dist, palets))
  big_thumb = sorted(dists)[0][1]

  # находим мизинец
  dists = []
  for palets in paltsi:
    dist = np.sqrt((base_dict[3][0] - palets[0])**2 + (base_dict[3][1] - palets[1])**2)
    dists.append((dist, palets))
  mizinec = sorted(dists)[0][1]

  # находим безымянный
  dists = []
  for palets in paltsi:
    dist = np.sqrt((mizinec[0] - palets[0])**2 + (mizinec[1] - palets[1])**2)
    if dist != 0:
      dists.append((dist, palets))
  bezimyaniy = sorted(dists)[0][1]

  # находим указательный
  dists = []
  for palets in paltsi:
    if palets != mizinec:
      dist = np.sqrt((big_thumb[0] - palets[0])**2 + (big_thumb[1] - palets[1])**2)
      if dist != 0:
        dists.append((dist, palets))
  ukazatelniy = sorted(dists)[0][1]

  # находим средний путём исключения
  for palets in paltsi:
    if palets != big_thumb and palets != ukazatelniy and palets != mizinec and palets != bezimyaniy:
      sredniy = palets
  
  # словарь пальцев
  hands_dict = {0: big_thumb, 1: ukazatelniy, 2: sredniy, 3: bezimyaniy, 4: mizinec}

  # подсчет углов между пальцами
  angles = []
  for base, hand in enumerate(range(len(hands_dict)-1)):
    vec1 = np.array(hands_dict[hand]) - np.array(base_dict[base])
    vec2 = np.array(hands_dict[hand+1]) - np.array(base_dict[base])
    angles.append(angle_between(vec1, vec2))
  # print(angles)

  # подсчет результата
  s = ''
  for i in range(len(angles)):
    # если палец большой
    if i == 0:
      if angles[i] < 27.5:
        s += f'{i+1}+'
      else:
        s += f'{i+1}-'
    elif i == 3:
      if angles[i] < 20:
        s += f'{i+1}+'
      else:
        s += f'{i+1}-'
    else:
      if angles[i] < 16.5:
          s += f'{i+1}+'
      else:
          s += f'{i+1}-'
  s += '5'

  image_rgb = cv.putText(image_rgb, s, (10,40), cv.FONT_HERSHEY_PLAIN, 3, (251, 255, 0), 2, cv.LINE_AA)
  cv.line(image_rgb, big_thumb, base_dict[0], (0,255,0), 2)
  cv.line(image_rgb, base_dict[0], ukazatelniy, (0,255,0), 2)
  cv.line(image_rgb, ukazatelniy, base_dict[1], (0,255,0), 2)
  cv.line(image_rgb, base_dict[1], sredniy, (0,255,0), 2)
  cv.line(image_rgb, sredniy, base_dict[2], (0,255,0), 2)
  cv.line(image_rgb, base_dict[2], bezimyaniy, (0,255,0), 2)
  cv.line(image_rgb, bezimyaniy, base_dict[3], (0,255,0), 2)
  cv.line(image_rgb, base_dict[3], mizinec, (0,255,0), 2)
  cv.imwrite('res/res_' + name[:-4] + '.png', cv.cvtColor(image_rgb, cv.COLOR_BGR2RGB))

  plt.figure(figsize=(8, 6))
  plt.title('Результат для ' + name, fontsize=14)
  plt.imshow(image_rgb)
  plt.axis("off")
  plt.show()
  return hands_dict, base_dict, s

# Основная часть

Для начала загрузим обученную модель

P.S.: здесь дополнительно скачиваются данные размером около 500MB, поэтому эта ячейка может выполняться дольше обычного (если совсем долго, можно попробовать перезапустить ядро)

P.S.S: в GoogleColab она выполняется быстро :)

In [ ]:
model = UNet()
model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))

Считаем названия всех файлов из папки Samples

In [11]:
PATH = 'Samples'
tmp = sorted(os.listdir(PATH))
names = []
for name in tmp:
    if name.endswith(".tif"):
        names.append(name)

По желанию, можно вывести исходные изображения, запустив ячейку ниже

In [ ]:
for name in names:
    image = cv.imread(PATH + '/' + name)
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.title(name, fontsize=14)
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.show()

Узнаем результат!

In [ ]:
f = open('Results.txt', 'w')
for name in names:
  res = predict(model, PATH, name)
  if res != -1:
    f.write(res[2] + '\n')
    s = '!,' + name + ','
    for hand in res[0].values():
      s += 'T ' + str(hand[0]) + ' ' + str(hand[1]) + ','
    for base in res[1].values():
      s += 'V ' + str(base[0]) + ' ' + str(base[1]) + ','
    s += '?'
    f.write(s + '\n')
f.close()